In [1]:
import pydeck as pdk

# pydeck: High scale geospatial visualization in Python

In [2]:
import geopandas as gpd
import pandas as pd

UK_ACCIDENTS_DATA = (
    "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/uk-accidents-2014.csv"
)

df = pd.read_csv(UK_ACCIDENTS_DATA)
df.head()

,lng,lat
0,-0.198465,51.505538
1,-0.178838,51.491836
2,-0.205590,51.514910
3,-0.208327,51.514952
4,-0.206022,51.496572


## Basic example: Plotting a Scatter Plot

In [3]:
import pydeck as pdk

UK_ACCIDENTS_DATA = (
    "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/uk-accidents-2014.csv"
)


df = pd.read_csv(UK_ACCIDENTS_DATA)

# Define a layer to display on a map
layer = pdk.Layer(
    "ScatterplotLayer",
    df,
    get_position=["lng", "lat"],
    # You can see the the deck.gl ScatterplotLayer docs for more info:
    # https://deck.gl/docs/api-reference/layers/scatterplot-layer
    auto_highlight=True,
    # Smallest a point is allowed to get–here, at least one pixel
    radius_min_pixels=1,
    # RGBA color value of point's fill
    get_fill_color=[255, 139, 51, 140],
    # Radius of the point in meters
    get_radius=100,
)

# Set the viewport location
## You can use a tool like https://duberste.in/viewport_finder/ to find this quickly
view_state = pdk.ViewState(
    longitude=-1.415, latitude=52.2323, zoom=6, min_zoom=5, max_zoom=15, pitch=40.5, bearing=-27.36,
)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style=pdk.map_styles.DARK_NO_LABELS,
)
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": -27.36, "latitude": 52.2323, "…

## Aggregating to Hexagon

In [4]:
import pydeck as pdk

# Same data set, read directly from a URL
UK_ACCIDENTS_DATA = (
    "https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/uk-accidents-2014.csv"
)

# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    UK_ACCIDENTS_DATA,
    get_position=["lng", "lat"],
    # You can see the the deck.gl HexagonLayer docs for more info:
    # https://deck.gl/docs/api-reference/aggregation-layers/hexagon-layer
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1,
)

# Set the viewport location
## You can use a tool like https://duberste.in/viewport_finder/ to find this quickly
view_state = pdk.ViewState(
    longitude=-1.415,
    latitude=52.2323,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36,
)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    map_style=pdk.map_styles.DARK_NO_LABELS,
    tooltip={'text': 'Number of observations: {elevationValue}'}
)
r.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"bearing": -27.36, "latitude": 52.2323, "…

## Plotting GeoJSON with geopandas

In [5]:
HURRICANES_URL = 'https://raw.githubusercontent.com/ajduberstein/geo_datasets/master/historical-hurricanes.geo.json'
gdf = gpd.read_file(HURRICANES_URL)
gdf.head()

,basin,number,year,name,entries,maximum_sustained_wind,month,day,hour,minute,record_identifier,status_of_system,latitude,longitude,minimum_pressure,geometry
0,AL,01,1851,UNNAMED,14,80,06,25,00,00,,HU,28.0,-94.8,-999,"LINESTRING (-94.80000 28.00000, -95.40000 28.0..."
1,AL,01,1851,UNNAMED,14,80,06,25,06,00,,HU,28.0,-95.4,-999,"LINESTRING (-95.40000 28.00000, -96.00000 28.0..."
2,AL,01,1851,UNNAMED,14,80,06,25,12,00,,HU,28.0,-96.0,-999,"LINESTRING (-96.00000 28.00000, -96.50000 28.1..."
3,AL,01,1851,UNNAMED,14,80,06,25,18,00,,HU,28.1,-96.5,-999,"LINESTRING (-96.50000 28.10000, -96.80000 28.2..."
4,AL,01,1851,UNNAMED,14,80,06,25,21,00,L,HU,28.2,-96.8,-999,"LINESTRING (-96.80000 28.20000, -97.00000 28.2..."


In [6]:
# You can plot GeoJSON via GeoPandas, without specifying a position column
layers = pdk.Layer(
    "GeoJsonLayer",
    data=gdf,
    pickable=True,  # Data can be selected
    stroked=False,  # Data shouldn't have an outline
    filled=True,  # Polygons should be filled
    line_width_scale=20,
    line_width_min_pixels=2,
    get_line_color=[200, 160, 180, 200],
    get_radius=10,
    get_line_width=1,
)
deck = pdk.Deck(layers, tooltip={'text': '{name} in {year}'})

deck.show()

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 0, "longitude": 0, "zoom": 1}…

## Plotting data over time

In [7]:
import time
import ipywidgets

# You can plot GeoJSON via GeoPandas, without specifying a position column
layer = pdk.Layer(
    "GeoJsonLayer",
    data=gdf,
    pickable=True,  # Data can be selected
    stroked=False,  # Data shouldn't have an outline
    filled=True,    # Polygons should be filled
    line_width_scale=20,
    line_width_min_pixels=2,
    get_line_color=[200, 160, 180, 200],
    get_radius=10,
    get_line_width=1,
)
deck = pdk.Deck(layer, tooltip={'text': '{name} in {year}'})

year = ipywidgets.HTML()
display(year)
display(deck.show())

for i in range(int(gdf.year.min()), int(gdf.year.max()) + 1):
    deck.layers[0].data = gdf[gdf['year'] == str(i)]
    deck.update()
    year.value = f'<b>{i}</b>'
    time.sleep(0.1)

HTML(value='')

DeckGLWidget(google_maps_key=None, json_input='{"initialViewState": {"latitude": 0, "longitude": 0, "zoom": 1}…